In [1]:
!date

Tue Apr 27 11:45:40 PDT 2021


# Install packages and download data

In [7]:
import sys
COLAB = "google.colab" in sys.modules


In [1]:
if COLAB:
    !git clone https://github.com/sbooeshaghi/BMGP_2020.git

Cloning into 'BMGP_2020'...
remote: Enumerating objects: 98, done.
remote: Counting objects: 100% (98/98), done.
remote: Compressing objects: 100% (81/81), done.
remote: Total 98 (delta 24), reused 77 (delta 10), pack-reused 0
Unpacking objects: 100% (98/98), done.


In [2]:
!pip --quiet install CITE-seq-Count==1.4.4

In [4]:
if COLAB:
    !pip install --quiet \
    kb-python==0.26.0 \
    anndata==0.7.6 \
    CITE-seq-Count==1.4.4

In [7]:
if COLAB:
    # download the relevant data
    !wget -O clicktag_fastqs.tar.gz \
    -q --show-progress --progress=bar:force  \
    https://caltech.box.com/shared/static/zqaom7yuul7ujetqyhnd4lvf8vhzqsyg.gz \
    2>&1
    !tar -xf clicktag_fastqs.tar.gz

clicktag_fastqs.tar 100%[===================>]   1.47G  12.7MB/s    in 89s     


# Preprocess with kb

In [9]:
%%time
!INDEX="BMGP_2020/references/clicktag/kite/features.idx" && \
F2B="BMGP_2020/references/clicktag/kite/f2b.txt" && \
FASTA="BMGP_2020/references/clicktag/kite/features.fa" && \
FB="BMGP_2020/references/clicktag/kite/feature_barcodes.txt" && \
kb ref \
-i $INDEX \
-g $F2B \
-f1 $FASTA \
--workflow kite \
$FB

[2021-04-29 11:28:29,716]    INFO Generating mismatch FASTA at BMGP_2020/references/clicktag/kite/features.fa
[2021-04-29 11:28:29,763]    INFO Creating transcript-to-gene mapping at BMGP_2020/references/clicktag/kite/f2b.txt
[2021-04-29 11:28:29,781]    INFO Skipping kallisto index because BMGP_2020/references/clicktag/kite/features.idx already exists. Use the --overwrite flag to overwrite.
CPU times: user 123 ms, sys: 32 ms, total: 155 ms
Wall time: 5.09 s


## No mismatches comparison

In [4]:
%%time
#FB="BMGP_2020/references/clicktag/kite_no_mismatches/feature_barcodes.txt" && \
!INDEX="../../../references/clicktag/kite_no_mismatches/features.idx" && \
F2B="../../../references/clicktag/kite_no_mismatches/f2b.txt" && \
FASTA="../../../references/clicktag/kite_no_mismatches/features.fa" && \
FB="../../../references/clicktag/kite_no_mismatches/feature_barcodes.txt" && \
kb ref \
-i $INDEX \
-g $F2B \
-f1 $FASTA \
--workflow kite \
--no-mismatches \
$FB

[2021-06-11 12:30:59,081]    INFO Generating mismatch FASTA at ../../../references/clicktag/kite_no_mismatches/features.fa
[2021-06-11 12:30:59,088]    INFO Creating transcript-to-gene mapping at ../../../references/clicktag/kite_no_mismatches/f2b.txt
[2021-06-11 12:30:59,089]    INFO Indexing ../../../references/clicktag/kite_no_mismatches/features.fa to ../../../references/clicktag/kite_no_mismatches/features.idx
CPU times: user 25 ms, sys: 6.84 ms, total: 31.8 ms
Wall time: 1.63 s


In [18]:
%%bash
FB="BMGP_2020/references/clicktag/kite/feature_barcodes.txt" && \
INDEX="BMGP_2020/references/clicktag/kite/features.idx" && \
F2B="BMGP_2020/references/clicktag/kite/f2b.txt" && \
FASTA="BMGP_2020/references/clicktag/kite/features.fa" && \
OUT="kite_out" && \
FASTQS=$(FASTQDIR="fastqs/" && paste -d" " \
<(ls $FASTQDIR | awk -v p=$FASTQDIR '{print p$0}' | grep R1) \
<(ls $FASTQDIR | awk -v p=$FASTQDIR '{print p$0}' | grep R2)) && \
kb count \
-i $INDEX \
-g $F2B \
-x 10xv2 \
-o $OUT \
--workflow kite \
--h5ad \
--filter bustools \
--overwrite \
$FASTQS

[2021-04-29 13:22:47,405]    INFO Using index BMGP_2020/references/clicktag/kite/features.idx to generate BUS file to kite_out from
[2021-04-29 13:22:47,406]    INFO         fastqs/NSC_Fixed_S2_L001_R1_001.fastq.gz
[2021-04-29 13:22:47,406]    INFO         fastqs/NSC_Fixed_S2_L001_R2_001.fastq.gz
[2021-04-29 13:24:52,736]    INFO Sorting BUS file kite_out/output.bus to kite_out/tmp/output.s.bus
[2021-04-29 13:25:16,124]    INFO Whitelist not provided
[2021-04-29 13:25:16,125]    INFO Copying pre-packaged 10XV2 whitelist to kite_out
[2021-04-29 13:25:16,307]    INFO Inspecting BUS file kite_out/tmp/output.s.bus
[2021-04-29 13:25:26,103]    INFO Correcting BUS records in kite_out/tmp/output.s.bus to kite_out/tmp/output.s.c.bus with whitelist kite_out/10xv2_whitelist.txt
[2021-04-29 13:25:38,368]    INFO Sorting BUS file kite_out/tmp/output.s.c.bus to kite_out/output.unfiltered.bus
[2021-04-29 13:25:58,192]    INFO Generating count matrix kite_out/counts_unfiltered/cells_x_features from B

In [8]:
%%bash
FB="../../../references/clicktag/kite_no_mismatches/feature_barcodes.txt" && \
INDEX="../../../references/clicktag/kite_no_mismatches/features.idx" && \
F2B="../../../references/clicktag/kite_no_mismatches/f2b.txt" && \
FASTA="../../../references/clicktag/kite_no_mismatches/features.fa" && \
OUT="kite_no_mismatches" && \
FASTQS=$(FASTQDIR="fastqs/" && paste -d" " \
<(ls $FASTQDIR | awk -v p=$FASTQDIR '{print p$0}' | grep R1) \
<(ls $FASTQDIR | awk -v p=$FASTQDIR '{print p$0}' | grep R2)) && \
kb count \
-i $INDEX \
-g $F2B \
-x 10xv2 \
-o $OUT \
--workflow kite \
--h5ad \
--filter bustools \
--overwrite \
$FASTQS

[2021-06-11 12:39:31,663]    INFO Using index ../../../references/clicktag/kite_no_mismatches/features.idx to generate BUS file to kite_no_mismatches from
[2021-06-11 12:39:31,663]    INFO         fastqs/NSC_Fixed_S2_L001_R1_001.fastq.gz
[2021-06-11 12:39:31,663]    INFO         fastqs/NSC_Fixed_S2_L001_R2_001.fastq.gz
[2021-06-11 12:40:10,835]    INFO Sorting BUS file kite_no_mismatches/output.bus to kite_no_mismatches/tmp/output.s.bus
[2021-06-11 12:40:19,377]    INFO Whitelist not provided
[2021-06-11 12:40:19,378]    INFO Copying pre-packaged 10XV2 whitelist to kite_no_mismatches
[2021-06-11 12:40:19,468]    INFO Inspecting BUS file kite_no_mismatches/tmp/output.s.bus
[2021-06-11 12:40:22,750]    INFO Correcting BUS records in kite_no_mismatches/tmp/output.s.bus to kite_no_mismatches/tmp/output.s.c.bus with whitelist kite_no_mismatches/10xv2_whitelist.txt
[2021-06-11 12:40:26,240]    INFO Sorting BUS file kite_no_mismatches/tmp/output.s.c.bus to kite_no_mismatches/output.unfiltered

# Preprocess with cite-seq-count

In [19]:
!gunzip BMGP_2020/references/whitelist_10xv2.txt.gz

In [31]:
!R1="fastqs/NSC_Fixed_S2_L001_R1_001.fastq.gz" && \
R2="fastqs/NSC_Fixed_S2_L001_R2_001.fastq.gz" && \
FB="BMGP_2020/references/clicktag/cite_seq_barcodes.csv" && \
WL="BMGP_2020/references/whitelist_10xv2.txt" && \
/usr/bin/time -v CITE-seq-Count -R1 $R1  -R2 $R2 --no_umi_correction -t $FB -wl $WL -cbf 1 -cbl 16 -umif 17 -umil 26 -cells 21000 --start-trim 28  -o ./cite_seq_out

Loading whitelist
Counting number of reads
Started mapping
Processing 15,928,888 reads
CITE-seq-Count is running with 48 cores.
Mapping done for process 48987. Processed 331,851 reads
Mapping done for process 48988. Processed 331,851 reads
Mapping done for process 48990. Processed 331,851 reads
Mapping done for process 48989. Processed 331,851 reads
Mapping done for process 48991. Processed 331,851 reads
Mapping done for process 48992. Processed 331,851 reads
Mapping done for process 48993. Processed 331,851 reads
Mapping done for process 48994. Processed 331,851 reads
Mapping done for process 48995. Processed 331,851 reads
Mapping done for process 48996. Processed 331,851 reads
Mapping done for process 48997. Processed 331,851 reads
Mapping done for process 48998. Processed 331,851 reads
Mapping done for process 48999. Processed 331,851 reads
Mapping done for process 49000. Processed 331,851 reads
Mapping done for process 49001. Processed 331,851 reads
Mapping done for process 49002. 